In [1]:
import sys
sys.path.append('..')

In [4]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import initializers
from preprocessing import parse_aug_fn, parse_fn

In [5]:
train_split, valid_split = ['train[:90%]', 'train[90%:]']
train_data, info = tfds.load('cifar10', split=train_split, with_info=True)
vaild_data = tfds.load('cifar10', split=valid_split)
test_data = tfds.load('cifar10', split='test')

print(train_data)

<PrefetchDataset shapes: {id: (), image: (32, 32, 3), label: ()}, types: {id: tf.string, image: tf.uint8, label: tf.int64}>


In [6]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
batch_size = 64
train_num = int(info.splits['train'].num_examples / 10 )* 9

train_data = train_data.shuffle(train_num)
train_data = train_data.map(map_func=parse_aug_fn, num_parallel_calls=AUTOTUNE)
train_data = train_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

vaild_data = vaild_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
vaild_data = vaild_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)

test_data = test_data.map(map_func=parse_fn, num_parallel_calls=AUTOTUNE)
test_data = test_data.batch(batch_size).prefetch(buffer_size=AUTOTUNE)


In [7]:
#使用高階API訓練模型
inputs = keras.Input(shape=(32, 32, 3))
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(inputs)
x = layers.MaxPool2D()(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(256, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(128, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Conv2D(64, 3, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu', kernel_initializer='glorot_uniform')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

model_1 = keras.Model(inputs=inputs, outputs=outputs)
model_1.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 13, 13, 128)       73856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 256)       295168    
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 9, 9, 128)         295040    
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 7, 7, 64)         

In [8]:
log_dirs = 'lab6-logs'
model_cbk = keras.callbacks.TensorBoard(log_dirs)
model_dirs = log_dirs + '/models'
os.makedirs(model_dirs, exist_ok=True)
save_model = keras.callbacks.ModelCheckpoint(model_dirs + '/save.h5',monitor='val_root_mean_squared_error', mode='max')


In [9]:
model_1.compile(keras.optimizers.Adam(), loss=keras.losses.CategoricalCrossentropy(from_logits=True), metrics=[keras.metrics.CategoricalAccuracy()])
history = model_1.fit(train_data, epochs=100, validation_data=vaild_data, callbacks=[model_cbk, save_model])

Epoch 1/100
  1/704 [..............................] - ETA: 0s - loss: 2.3017 - categorical_accuracy: 0.1250WARNING:tensorflow:From c:\Users\Admin\miniconda3\envs\tf2\lib\site-packages\tensorflow\python\ops\summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.


Instructions for updating:
use `tf.profiler.experimental.stop` instead.


704/704 [==============================] - 55s 77ms/step - loss: 2.0074 - categorical_accuracy: 0.2492 - val_loss: 1.7715 - val_categorical_accuracy: 0.3748
Epoch 2/100
704/704 [==============================] - 57s 81ms/step - loss: 1.7392 - categorical_accuracy: 0.3686 - val_loss: 1.4586 - val_categorical_accuracy: 0.4824
Epoch 3/100
704/704 [==============================] - 57s 81ms/step - loss: 1.5934 - categorical_accuracy: 0.4256 - val_loss: 1.3131 - val_categorical_accuracy: 0.5194
Epoch 4/100
704/704 [==============================] - 57s 82ms/step - loss: 1.5027 - categorical_accuracy: 0.4616 - val_loss: 1.1849 - val_categorical_accuracy: 0.5788
Epoch 5/100
704/704 [==============================] - 57s 81ms/step - loss: 1.4223 - categorical_accuracy: 0.4952 - val_loss: 1.1555 - val_categorical_accuracy: 0.5982
Epoch 6/100
704/704 [==============================] - 58s 83ms/step - loss: 1.3574 - categorical_accuracy: 0.5241 - val_loss: 1.0840 - val_categorical_accuracy: 0.609

In [ ]:
#客製化模型
class CustomConv2D(tf.keras.layers.Layer):
    def __init__(self, filters, kernel_size, strides=(1, 1), padding='valid'):
        super(CustomConv2D, self).__init__()
        self.filters = filters
        self.kernel_size = kernel_size
        self.strides = strides
        self.padding = padding

    def build(self, input_shape):
        kernel_h, kernel_w = self.kernel_size
        input_dim = input_shape[-1]
        self.w = self.add_weight(name='kernel', shape=(kernel_h, kernel_w, input_dim, self.filters), initializer='glorot_uniform', trainable=True)
        self.b = self.add_weight(name='bias', shape=(self.filters,), initializer='zeros', trainable=True)

    def call(self, inputs):
        return tf.nn.relu(tf.nn.conv2d(inputs, self.w, self.strides, self.padding) + self.b)

In [ ]:
inputs = keras.Input(shape=(32, 32, 3))
x = CustomConv2D(64, (3, 3))(inputs)
x = layers.MaxPool2D()(x)
x = CustomConv2D(128, (3, 3))(x)
x = CustomConv2D(256, (3, 3))(x)
x = CustomConv2D(128, (3, 3))(x)
x = CustomConv2D(64, (3, 3))(x)
x = layers.Flatten()(x)
x = layers.Dense(64, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

model_2 = keras.Model(inputs=inputs, outputs=outputs)
model_2.summary()

In [ ]:
class SaveModel(keras.callbacks.Callback):
    def __init__(self, weightS_file, monitor='loss', mode='min',save_weights_only=False):
        super(SaveModel, self).__init__()
        self.weightS_file = weightS_file
        self.monitor = monitor
        self.mode = mode
        self.save_weights_only = save_weights_only
        if mode == 'min':
            self.best = np.Inf
        else:
            self.best = -np.Inf

    def save_model(self):
        if self.save_weights_only:
            self.model.save_weights(self.weightS_file)
        else:
            self.model.save(self.weightS_file)        

    def on_epoch_end(self, epoch, logs=None):
        monitor_value = logs.get(self.monitor)
        if self.mode == 'min' and monitor_value < self.best:
            self.best = monitor_value
            self.save_model()
        elif self.mode == 'max' and monitor_value > self.best:
            self.best = monitor_value
            self.save_model()
    

In [ ]:
log_dirs = 'lab6-logs'
model_cbk = keras.callbacks.TensorBoard(log_dirs)
model_dirs = log_dirs + '/models'
os.makedirs(model_dirs, exist_ok=True)
save_model = keras.callbacks.ModelCheckpoint(model_dirs + '/custom_save.h5',monitor='val_root_mean_squared_error', mode='max',save_weights_only=True)


In [ ]:
def custom_categorical_crossentropy(y_true, y_pred):
    return tf.nn.softmax_cross_entropy_with_logits(label=y_true, logits=y_pred)

def CustomCategoryAccuracy(tf.keras.metrics.Metric):
    def __init__(self, name='custom_catrgory_accuracy', **kwargs):
        super(CustomCategoryAccuracy, self).__init__(name=name, **kwargs)
        self.correct = self.add_weight(name='correct_numbers', initializer='zeros')
        self.total = self.add_weight(name='total_numbers', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.argmax(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        values = tf.cast(y_true == y_pred, 'float32')
        values_sum = tf.reduce_sum(values)
        num_values = tf.cast(tf.size(values), 'float32')
        self.correct.assign_add(values)
        self.total.assign_add(num_values)

    def result(self):
        return tf.math.divide_no_nan(self.correct, self.total)

    def reset_states(self):
        self.correct.assign(0.)
        self.total.assign(0.)

In [ ]:
model_2.compile(keras.optimizers.Adam(), loss=custom_categorical_crossentropy, metrics=[CustomCategoryAccuracy()])
history = model_2.fit(train_data, epochs=100, validation_data=vaild_data, callbacks=[model_cbk, save_model])

In [ ]:
model_1 = keras.models.load_model(model_dirs + '/save.h5')
model_2 = keras.models.load_model(model_dirs + '/custom_save.h5')
loss_1, acc_1 = model_1.evaluate(test_data)
loss_2, acc_2 = model_2.evaluate(test_data)
loss=[loss_1, loss_2]
acc=[acc_1, acc_2]
pd.DataFrame({'loss':loss, 'accuracy':acc}, index=['model_1', 'model_2'])